In [21]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [23]:
import psycopg2 as psyco
from bs4 import BeautifulSoup
import requests as req

In [24]:
routenumbers = [104, 105, 108, 109, 111, 113, 114, 115, 116, 117, 118, 122, 126, 127, 132, 133, 141, 142, 143] 
idnumbers = [501, 521, 522, 502, 393, 503, 504, 524, 525, 512, 527, 528, 733, 513, 509, 531, 532, 533, 534]
paths = [str(routenumbers[i])+'/'+str(idnumbers[i])+'#edit-group-stop-list' for i in range(len(routenumbers))]

In [25]:
filepath = 'https://transportation.umd.edu/shuttle-um/{}'
souplist = []
stopdict = dict()
routenames = {}
for i in range(len(routenumbers)):
    #Create beautifulsoup object representing route page, append to soup object list
    url = req.get(filepath.format(paths[i])).text
    soup = BeautifulSoup(url, 'html.parser')
    soup.prettify()
    souplist.append(soup)

In [26]:
for i in range(len(souplist)):
    ##Access current soup object
    soupobj = souplist[i]
    
    #Scrape routenames
    routenames[routenumbers[i]] = soupobj.find("h1", class_="page-title").find("span").text
    
    #Scrape stops on route
    for j in soupobj.find('tbody').findAll('td')[2::3]:
        j = j.text.replace(u'\xa0', u' ').strip()
        if(j not in stopdict):
            stopdict[j] = set()
        stopdict[j].add(routenumbers[i])

In [38]:
print(routenames)
routenames[141]=routenames[141].replace('&','')
routenames[142]=routenames[142].replace('&','')

{104: 'College Park Metro', 105: 'Campus Parking Connector', 108: 'Adelphi', 109: 'River Road', 111: 'Silver Spring', 113: 'Hyattsville', 114: 'University View', 115: 'Orange', 116: 'Purple', 117: ' Blue', 118: 'Gold', 122: 'Green', 126: 'New Carrollton', 127: 'Mazza GrandMarc', 132: 'The Varsity', 133: 'Grocery Shopping Shuttle', 141: 'Gaithersburg Park  Ride', 142: 'Columbia Park  Ride', 143: 'Greenbelt'}


In [58]:
conn = psyco.connect(dbname="so-basicly-1217.defaultdb", host = 'free-tier14.aws-us-east-1.cockroachlabs.cloud', user="snesun", password="GPmRJ87pVWztddyVQ9L0rQ", port = 26257)
with conn:
    with conn.cursor() as cur:
        cur = conn.cursor()
        cur.execute("CREATE TABLE ShuttleStops(ID INT NOT NULL PRIMARY KEY, Name VARCHAR(100), Routes VARCHAR(100));")
        cur.execute("CREATE TABLE Route(ID INT NOT NULL PRIMARY KEY, Name VARCHAR(100));")
        for route in routenumbers:
           cur.execute("CREATE TABLE {}(ID INT NOT NULL PRIMARY KEY, Outbound BOOLEAN, Time VARCHAR(10), Day VARCHAR(10));".format(routenames[route].replace(" ","")))

        ShuttleStops
        i = 1
        for key in stopdict:
            cur.execute("INSERT INTO ShuttleStops(ID, Name, Routes) VALUES({0},{1},{2})".format(str(i),key.replace(" ",""), "".join(list(str(val) for val in value))))
            i += 1
        cur.execute("SHOW TABLES;")
        print(cur.fetchall())
        
        Routes
        for key,value in routenames.items():
            cur.execute("INSERT INTO Route VALUES({},{});".format(key,value.replace(" ","")))
        
        conn.commit()
        cur.close()

UndefinedColumn: column "collegeparkmetro" does not exist
